In [0]:
from bs4 import BeautifulSoup
from time import sleep
import requests
import re
import pandas as pd
from pandas import DataFrame
import numpy as np
import lxml.html

In [0]:
extract_song_number = re.compile("[0-9]{1,}")

def crawler():
    artist_number = int(input("아티스트 번호를 입력하세요."))
    song_page = int(input("아티스트의 노래 페이지 수를 입력하세요."))
    d = webdriver.Chrome("D:\chromedriver.exe")
    
    total_title_list = []
    total_song_number_list = []
    lyrics_list = []
    
    for page_nubmer in range(1, song_page+1):
        d.implicitly_wait(3)
        url_s = 'https://music.naver.com/artist/track.nhn?artistId='+str(artist_number)+'&page=' + str(page_nubmer)
        d.get(url_s)
        
        html = d.page_source
        soup = BeautifulSoup(html, 'html.parser')
        title = soup.select('.track')
        temp_title_list = [i.text.strip() for i in title]
        temp_song_number_list = [extract_song_number.findall(i.get('onclick'))[0] for i in title]
        
        total_title_list += temp_title_list
        total_song_number_list += temp_song_number_list
        
    for song_number in total_song_number_list:
        url2 = 'https://music.naver.com/lyric/index.nhn?trackId=' + str(song_number)
        response = requests.get(url2)
        html = response.text 
        soup = BeautifulSoup(html, 'html.parser')
        lyrics = soup.select('.section_lyrics')
        temp_lyrics = str(lyrics[0]).replace("""<div class="section_lyrics">\n<div class="show_lyrics" id="lyricText">""", "").replace("</div>\n</div>", "")
        temp_lyrics2 = temp_lyrics.strip().replace("<br/>", "\n")
        temp_lyrics3 = re.sub('\n\n\n+','\n',temp_lyrics2)
        lyrics_list.append(temp_lyrics3)
    
    df = pd.DataFrame({"artist_number" : [artist_number for _ in range(len(total_title_list))],
                 "title" : total_title_list, "lyrics" : lyrics_list})
    
    df.drop_duplicates("title", inplace = True)
    df.drop_duplicates("lyrics", inplace = True)
    
    df.drop(index = df[df.lyrics.apply(lambda x : 1 if """<div class="section_lyrics">""" in x else 0) == 1].index, inplace = True)
    df.index = np.arange(len(df))
    
    return df

In [0]:
df = crawler()

In [0]:
df.to_csv("김광석.csv", index = False, encoding = 'utf-8-sig')